# Using FairMedFM for SegFMs

This notebook further explores the internal modules of FairMedFM using segmentation. We have four modules, model, usage wrapper, dataloader, trainer, evaluation.

# Install FairMedFM

In [1]:
!git clone https://github.com/FairMedFM/FairMedFM

Cloning into 'FairMedFM'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 479 (delta 69), reused 86 (delta 39), pack-reused 330 (from 1)
Receiving objects: 100% (479/479), 3.18 MiB | 7.53 MiB/s, done.
Resolving deltas: 100% (257/257), done.


In [2]:
%cd FairMedFM

/content/FairMedFM


In [3]:
%mkdir data
%cd data
!wget https://object-arbutus.cloud.computecanada.ca/rjin/FairMedFM/TUSC.zip
!unzip TUSC.zip
%cd ..

/content/FairMedFM/data
--2024-10-30 23:35:23--  https://object-arbutus.cloud.computecanada.ca/rjin/FairMedFM/TUSC.zip
Resolving object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)... 206.12.48.146
Connecting to object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)|206.12.48.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 139307660 (133M) [application/zip]
Saving to: ‘TUSC.zip’

TUSC.zip            100%[===================>] 132.85M   224MB/s    in 0.6s    

2024-10-30 23:35:24 (224 MB/s) - ‘TUSC.zip’ saved [139307660/139307660]

Archive:  TUSC.zip
   creating: TUSC/
  inflating: TUSC/.DS_Store          
  inflating: __MACOSX/TUSC/._.DS_Store  
   creating: TUSC/label/
   creating: TUSC/image/
  inflating: TUSC/meta.csv           
  inflating: TUSC/metadata.csv       
  inflating: TUSC/label/170_27.png   
  inflating: TUSC/label/25_46.png    
  inflating: TUSC/label/6_92.png     
  inflating: TUSC/label/45_

In [4]:
!wget https://object-arbutus.cloud.computecanada.ca:443/rjin/FairMedFM/pretrained.zip
!unzip pretrained.zip
!rm -f pretrained.zip

--2024-10-30 23:35:27--  https://object-arbutus.cloud.computecanada.ca/rjin/FairMedFM/pretrained.zip
Resolving object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)... 206.12.48.146
Connecting to object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)|206.12.48.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6610529563 (6.2G) [application/zip]
Saving to: ‘pretrained.zip’

pretrained.zip      100%[===================>]   6.16G   176MB/s    in 35s     

2024-10-30 23:36:02 (178 MB/s) - ‘pretrained.zip’ saved [6610529563/6610529563]

Archive:  pretrained.zip
   creating: pretrained/
   creating: pretrained/medclip-resnet/
  inflating: pretrained/medclip-resnet/pytorch_model.bin  
 extracting: pretrained/medclip-resnet/medclip-pretrained.zip  
   creating: pretrained/C2L/
  inflating: pretrained/C2L/res18_C2L.pth  
   creating: pretrained/mococxr/
  inflating: pretrained/mococxr/r8w-00001-v2.pth.tar  
   creating: 

In [5]:
!pip install icecream==2.1.3
!pip install SimpleITK==2.3.1
!pip install einops==0.8.0
!pip install ipdb==0.13.13
!pip install git+https://github.com/facebookresearch/segment-anything.git@6fdee8f2727f4506cfbbe553e23b895e2795658
!pip install timm==1.0.3
!pip install open-clip-torch==2.24.0
!pip install salesforce-lavis==1.0.2
!pip install git+https://github.com/openai/CLIP.git@a1d071733d7111c9c014f024669f959182114e33
!pip install MedCLIP==0.0.3
!pip install torchmetrics==1.4.0.post0
!pip install peft==0.11.1
!pip install git+https://github.com/transcendentsky/tutils.git@8b02b20c9caa5553c96a6413b739ac47385fd730
!pip install transformers==4.40.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 41.8 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Cloning https://github.com/facebookresearch/segment-anything.git (to revision 6fdee8f2727f4506cfbbe553e23b895e2795658) to /tmp/pip-req-build-wgx0kqt6
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-wgx0kqt6
  Running command git checkout -q 6fdee8f2727f4506cfbbe553e23b895e2795658
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e2795658
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36590 sha256=f432a268c74c1f20d0f0ea3672673e712730194cb425b33dffe182ca8a9b1a06
  Stored in directory: /tmp/pip-ephem-wheel-cache-kf8jv1b_/wheels/ed/f9/15/7c73be3b9a46fe7a94380213fa6e09fb5

  Cloning https://github.com/openai/CLIP.git (to revision a1d071733d7111c9c014f024669f959182114e33) to /tmp/pip-req-build-irqu6h6x
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-irqu6h6x
  Running command git rev-parse -q --verify 'sha^a1d071733d7111c9c014f024669f959182114e33'
  Running command git fetch -q https://github.com/openai/CLIP.git a1d071733d7111c9c014f024669f959182114e33
  Running command git checkout -q a1d071733d7111c9c014f024669f959182114e33
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369499 sha256=7a5be2c1cbe357c79a7108f130a7a751b517d35ee9629aa90b73a0499ac2f661
  Stored in directory: /root/.cache/pip/wheels/df/64/50/148b28b28a0f816d0e876207653618a4ec90e3a9c0f9c6e2a5
Successfully built clip
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━

In [6]:
!pip freeze

absl-py==1.4.0
accelerate==0.34.2
aiohappyeyeballs==2.4.0
aiohttp==3.10.5
aiosignal==1.3.1
alabaster==0.7.16
albucore==0.0.16
albumentations==1.4.15
altair==4.2.2
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.19.0
astropy==6.1.3
astropy-iers-data==0.2024.9.16.0.32.21
asttokens==2.4.1
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==24.2.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.17.0
bigquery-magics==0.2.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.4.3
bqplot==0.12.43
braceexpand==0.1.7
branca==0.7.2
build==1.2.2
CacheControl==0.14.0
cachetools==5.5.0
catalogue==2.0.10
certifi==2024.8.30
cffi==1.17.1
cfgv==3.4.0
chardet==3.0.4
charset-normalizer==3.3.2
chex==0.1.86
clarabel==0.9.0
click==8.1.7
clip @ git+https://github.com/openai/CLIP.git@a1d071733d7111c9c014f024669f959182114e33
cloudp

# Segmentation Example

In [7]:
!mkdir log

In [9]:
# setting experiment parameters
import os
os.chdir("/content/FairMedFM")
import json
from utils import basics
import torch

class Settings(object):
    def __init__(self) -> None:
        self.task = "seg"
        self.usage = "seg2d" # choose to use 2d or 3d model
        self.dataset = "TUSC"
        self.sensitive_name = "Sex"
        self.pos_class = 255 # the value of the gt mask
        self.model = "SAM" # choose from "SAM", "MedSAM", "SAMMed2D", "FT-SAM", "TinySAM", "MobileSAM"
        self.sam_ckpt_path = "./pretrained/SAM.pth"
        self.img_size = 1024
        self.prompt = "center" # type of prompt, choose from "bbox", "rand", "rands", "center"
        self.save_folder = "log"

        self.cuda = True
        self.device = torch.device("cuda" if self.cuda else "cpu")
        self.data_setting = self.get_data_settings()

        # the following attrs are used as a placeholder, not used for segmentation
        self.batch_size = 1
        self.method = "erm"
        self.augment = False
        self.random_seed = 0
        self.cls_balance = False
        self.total_epochs = 1

    def get_data_settings(self):
        with open(f"configs/datasets/{self.dataset}.json", "r") as f:
            data_setting = json.load(f)
            data_setting["augment"] = False
            data_setting["test_meta_path"] = data_setting[
                f"test_{str.lower(self.sensitive_name)}_meta_path"]

            if self.pos_class is not None:
                data_setting["pos_class"] = self.pos_class
        return data_setting

settings = Settings()
logger = basics.setup_logger(
    "segmentation", settings.save_folder, "history", screen=False, tofile=True)


## Get evaluate dataset (TUSC as an example)

In [10]:
from datasets.utils import get_dataset
test_data, test_dataloader, test_meta = get_dataset(settings, split="test")

print("The number of test samples: ", len(test_data))
print(test_meta.head())

loaded dataset  TUSC
The number of test samples:  860
   index          imagePath          labelPath     sex  age sexBinary
0      0   image/118_36.png   label/118_36.png  Female   44         F
1      1  image/184_239.png  label/184_239.png  Female   48         F
2      2    image/55_15.png    label/55_15.png  Female   40         F
3      3    image/23_63.png    label/23_63.png  Female   62         F
4      4    image/46_56.png    label/46_56.png  Female   38         F


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## Get Model (SAM with bbox prompt as an example)

In [11]:
from models.utils import get_model
from wrappers.utils import get_warpped_model
from trainers.utils import get_trainer
import os
import torch
model = get_model(settings).to(settings.device)
model = get_warpped_model(settings, model, test_data).to(settings.device)  # SAMLearner
print("2D SegFM using 1 bounding box prompt performance:")
trainer = get_trainer(settings, model, logger, test_dataloader)


/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/content/FairMedFM/models/sam_builder/build_tinysam.py:819: UserWarning: Overwriting tiny_vit_5m_224 in registry with models.sam_builder.build_tinysam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/content/FairMedFM/models/sam_builder/build_tinysam.py:819: UserWarning: Overwriting tiny_vit_11m_224 in registry with models.sam_builder.b

Weights of model: #SAM# is loaded!
2D SegFM using 1 bounding box prompt performance:


## Evaluate the test set

In [12]:
trainer.evaluate(test_dataloader, save_path=os.path.join(
    settings.save_folder, settings.prompt))

  0%|          | 0/860 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 860/860 [07:15<00:00,  1.97it/s]
INFO:segmentation:-----------------meta info-------------------
INFO:segmentation:metrics: mean_dice: 0.2601, min_dice: 0.2563, max_dice: 0.2800, delta_dice: 0.0237, skewness_dice: 1.0329, std_dice: 0.011

## Present Results

In [16]:
from matplotlib import pyplot as plt
import seaborn as sns
import pickle
sns.set_theme(
    context="talk",
    style="ticks",
)

with open("./log/center/metrics.pkl", "rb") as f:
    metrics = pickle.load(f)

metrics["metrics"]

categories = list(metrics["metrics"].keys())
values = list(metrics["metrics"].values())

plt.figure(figsize=(8, 5), dpi=300)
ax = sns.barplot(x=values, y=categories, orient='h',color='#FF6347')

for i in ax.patches:
    ax.text(i.get_width() + 0.15,
            i.get_y() + i.get_height() / 2,
            f'{i.get_width():.4f}',
            ha='center', va='center')

plt.title('Fairness Metrics')
plt.xlabel('Values')
plt.xlim(0, 1.3)
plt.ylabel('Metrics')
plt.tight_layout()
plt.show()